In [443]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop, SGD, Adadelta, Adamax, Adagrad, Nadam, Ftrl
from keras.layers import Dense, Input

In [444]:
df = pd.read_csv(r'C:\Users\marku\Desktop\ML\MLGit\datasets\titanic.csv')

In [445]:
# Sets null values equal to the median of that column
def handle_null_median(df):
    # Need to set inplace=True, so it doesn't create a copy of the dataframe. Tried without and this led to null-values not being removed
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    df['Age'].fillna(df['Age'].median(), inplace=True)

    df['Embarked'].fillna('S', inplace=True)

    return  df

df = handle_null_median(df)
df['Sex'] = df['Sex'].replace(['female', 'male'], [0,1])
df['FamilyMembersCount'] = df['Parch'] + df['SibSp'] + 1
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], axis=1)

In [446]:
def bins(df): # This is kind of feature engineering as well
    df['AgeGroup'] = 0
    df.loc[df['Age'] < 16.336, 'AgeGroup' ] = 0
    df.loc[(df['Age'] >= 16.336) & (df['Age'] < 32.252), 'AgeGroup'] = 1
    df.loc[(df['Age'] >= 32.252) & (df['Age'] < 48.168), 'AgeGroup'] = 2
    df.loc[(df['Age'] >= 48.168) & (df['Age'] < 64.084), 'AgeGroup'] = 3
    df.loc[df['Age'] >= 64.084, 'AgeGroup'] = 4

    df = df.drop('Age', axis=1)

    # Could also create bins for fare, but not sure Fare is needed.
    return df

df = bins(df)

df.head()

,Survived,Pclass,Sex,Fare,Embarked,FamilyMembersCount,AgeGroup
0,0,3,1,7.2500,S,2,1
1,1,1,0,71.2833,C,2,2
2,1,3,0,7.9250,S,1,1
3,1,1,0,53.1000,S,2,2
4,0,3,1,8.0500,S,1,2


In [447]:
features = df.drop('Survived',axis=1)
targets = df['Survived']

targets_onehot = pd.get_dummies(targets)
features_onehot = pd.get_dummies(features)
targets_onehot = targets_onehot.astype('float32')
features_onehot = features_onehot.astype('float32')

In [448]:
features_onehot = normalize(features_onehot)

train_features, test_features, train_targets, test_targets = train_test_split(features_onehot, targets_onehot, test_size=0.1, stratify=targets, random_state=42)

In [449]:
train_features_tensor = tf.convert_to_tensor(train_features)
test_features_tensor = tf.convert_to_tensor(test_features)
train_targets_tensor = tf.convert_to_tensor(train_targets)
test_targets_tensor = tf.convert_to_tensor(test_targets)

In [450]:
model = Sequential()

model.add(Input(8))
model.add(Dense(27, activation='relu'))
model.add(Dense(54, activation='relu'))
model.add(Dense(27, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

In [451]:
opt1 = Adam(learning_rate=0.01)
opt2 = RMSprop()
opt3 = SGD()
opt4 = Adadelta()
opt5 = Adamax()
opt6 = Adagrad()
opt7 = Nadam()
opt8 = Ftrl()
model.compile(optimizer=opt1, loss='binary_crossentropy', metrics=['accuracy'])

In [452]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=100, mode='auto')
model.fit(train_features_tensor, train_targets_tensor, epochs=250, batch_size=10, callbacks=[callback])

Epoch 1/250
81/81 [==============================] - 0s 2ms/step - loss: 0.6226 - accuracy: 0.6479
Epoch 2/250
81/81 [==============================] - 0s 1ms/step - loss: 0.5670 - accuracy: 0.6866
Epoch 3/250
81/81 [==============================] - 0s 1ms/step - loss: 0.5616 - accuracy: 0.7016
Epoch 4/250
81/81 [==============================] - 0s 2ms/step - loss: 0.5555 - accuracy: 0.7066
Epoch 5/250
81/81 [==============================] - 0s 2ms/step - loss: 0.5234 - accuracy: 0.7391
Epoch 6/250
81/81 [==============================] - 0s 1ms/step - loss: 0.5295 - accuracy: 0.7291
Epoch 7/250
81/81 [==============================] - 0s 1ms/step - loss: 0.5114 - accuracy: 0.7591
Epoch 8/250
81/81 [==============================] - 0s 1ms/step - loss: 0.5034 - accuracy: 0.7578
Epoch 9/250
81/81 [==============================] - 0s 2ms/step - loss: 0.5165 - accuracy: 0.7391
Epoch 10/250
81/81 [==============================] - 0s 1ms/step - loss: 0.5136 - accuracy: 0.7615
Epoch 11/

In [453]:
print(model.evaluate(train_features_tensor, train_targets_tensor), model.evaluate(test_features_tensor, test_targets_tensor))

3/3 [==============================] - 0s 2ms/step - loss: 0.6445 - accuracy: 0.7778
[0.3696633577346802, 0.8439450860023499] [0.6445497870445251, 0.7777777910232544]
